# 1. 유사도 계산하기

- 추천
    - 어떤 user와 취향이 유사한 user가 좋아하는 item 추천
    - 어떤 user가 좋아하는 item과 유사한 item을 찾음
- 사용자간 유사도 측정
    - 유클리드 거리: 거리가 멀수록 유사하지 않음
    - 피어슨상관계수: 두 객체 간의 공분산과 표준편차를 이용

In [1]:
import json
import numpy as np

## 1.1 Python code

### Euclidean score
![유클리드거리](uclidean_distance.png)

In [2]:
# Compute the Euclidean distance score between user1 and user2 
def euclidean_score(dataset, user1, user2):
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 두 사용자에게서 평점을 받은 영화 저장
    common_movies = {} 

    # 두 사용자에게서 모두 평점을 받은 영화 찾음
    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    # 사용자간 공통 영화가 없으면, score = 0 
    if len(common_movies) == 0:
        return 0

    squared_diff = [] 

    # 두 사용자에게서 평점을 받은 영화를 대상으로 사용자간의 평점 차이 값을 제곱
    for item in dataset[user1]:
        if item in dataset[user2]:
            squared_diff.append(np.square(dataset[user1][item] - dataset[user2][item]))
        
    # 유클리드 거리 값을 0 ~ 1로 변환
    return 1 / (1 + np.sqrt(np.sum(squared_diff))) 

### Pearson correlation score
![피어슨상관계수](peason_coefficient.png)

In [3]:
# Compute the Pearson correlation score between user1 and user2 
def pearson_score(dataset, user1, user2):
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 두 사용자에게서 평점을 받은 영화 저장
    common_movies = {}

    # 두 사용자에게서 모두 평점을 받은 영화 찾음
    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    num_ratings = len(common_movies) 

    # 사용자간 공통 영화가 없으면, score = 0 
    if num_ratings == 0:
        return 0

    # 사용자별로 공통으로 평가한 영화의 평점 합 계산 
    user1_sum = np.sum([dataset[user1][item] for item in common_movies])
    user2_sum = np.sum([dataset[user2][item] for item in common_movies])

    # 사용자별로 공통으로 평가한 영화의 평점의 제곱 합 계산
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in common_movies])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in common_movies])

    # 두 사용자가 평가한 영화 평점의 곱
    sum_of_products = np.sum([dataset[user1][item] * dataset[user2][item] for item in common_movies])

    # Calculate the Pearson correlation score
    Sxy = sum_of_products - (user1_sum * user2_sum / num_ratings)
    Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
    Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
    
    if Sxx * Syy == 0:
        return 0

    return Sxy / np.sqrt(Sxx * Syy)

In [4]:
ratings_file = 'ratings.json'

with open(ratings_file, 'r') as f:
    data = json.loads(f.read())
data

{'David Smith': {'Vertigo': 4,
  'Scarface': 4.5,
  'Raging Bull': 3.0,
  'Goodfellas': 4.5,
  'The Apartment': 1.0},
 'Brenda Peterson': {'Vertigo': 3.0,
  'Scarface': 1.5,
  'Raging Bull': 1.0,
  'Goodfellas': 2.0,
  'The Apartment': 5.0,
  'Roman Holiday': 4.5},
 'Bill Duffy': {'Vertigo': 4.5,
  'Scarface': 5.0,
  'Goodfellas': 4.5,
  'The Apartment': 1.0},
 'Samuel Miller': {'Scarface': 3.5,
  'Raging Bull': 5.0,
  'The Apartment': 1.0,
  'Goodfellas': 5.0,
  'Roman Holiday': 1.0},
 'Julie Hammel': {'Scarface': 2.5, 'Roman Holiday': 4.5, 'Goodfellas': 3.0},
 'Clarissa Jackson': {'Vertigo': 5.0,
  'Scarface': 4.5,
  'Raging Bull': 4.0,
  'Goodfellas': 2.5,
  'The Apartment': 1.0,
  'Roman Holiday': 1.5},
 'Adam Cohen': {'Vertigo': 3.5,
  'Scarface': 3.0,
  'The Apartment': 1.0,
  'Goodfellas': 4.5,
  'Roman Holiday': 3.0},
 'Chris Duncan': {'The Apartment': 1.5, 'Raging Bull': 4.5}}

In [5]:
user1 = "David Smith"
user2 = "Bill Duffy"

print("\nEuclidean score:")
print(euclidean_score(data, user1, user2))

print("\nPearson score:")
print(pearson_score(data, user1, user2))


Euclidean score:
0.585786437626905

Pearson score:
0.9909924304103233


## 1.2 scipy.spatial.distance 이용
- https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#module-scipy.spatial.distance
### cosine 유사도

In [6]:
from scipy.spatial.distance import cosine

In [7]:
a = np.asarray([3,4,3,1])
b = np.asarray([3,3,5,2])
c = np.asarray([3,5,4,1])

In [8]:
# cosine 유사도
round(1-cosine(a,c),4)

0.9941

## 1.3 numpy.linalg 모듈 이용
### SVD(Singular Value Decomposition): 특이값분해
![SVD](SVD.png)

In [9]:
from numpy import *
from numpy import linalg as la

In [10]:
def loadExData():
    return[[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

In [11]:
Data=loadExData()

In [12]:
U,Sigma,VT=linalg.svd(Data)

In [13]:
print(Sigma.round(4))

[9.7214 5.294  0.6842 0.     0.    ]


In [14]:
Sig2=mat([[Sigma[0], 0],[0, Sigma[1]]])
Sig2

matrix([[9.72140007, 0.        ],
        [0.        , 5.29397912]])

In [15]:
print(U[:,:2]*Sig2*VT[:2,:])

[[ 1.00497377e+00  1.00497377e+00  9.89899934e-01 -7.22620478e-04
  -7.22620478e-04]
 [ 2.00994753e+00  2.00994753e+00  1.97979987e+00 -1.44524096e-03
  -1.44524096e-03]
 [ 1.00497377e+00  1.00497377e+00  9.89899934e-01 -7.22620478e-04
  -7.22620478e-04]
 [ 5.02486883e+00  5.02486883e+00  4.94949967e+00 -3.61310239e-03
  -3.61310239e-03]
 [ 7.69884117e-01  7.69884117e-01  4.67288818e-01  2.03343270e+00
   2.03343270e+00]
 [ 1.41378969e-01  1.41378969e-01 -2.87093661e-01  2.97945956e+00
   2.97945956e+00]
 [ 4.71263231e-02  4.71263231e-02 -9.56978871e-02  9.93153188e-01
   9.93153188e-01]]


### 유사도 측정
- 유클리디언
- 피어슨
- 코사인

In [16]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [17]:
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

In [18]:
# inA가 3개 이상일 경우만 상관계수를 구할 수 있음
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

In [19]:
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [20]:
myMat = mat(loadExData2())
myMat

matrix([[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
        [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
        [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
        [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
        [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
        [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
        [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
        [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
        [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

In [21]:
ecludSim(myMat[:,0], myMat[:,4])

0.08462632608958592

In [22]:
ecludSim(myMat[:,0], myMat[:,0])

1.0

In [23]:
cosSim(myMat[:,0], myMat[:,4])

0.5

In [24]:
cosSim(myMat[:,0], myMat[:,0])

1.0

In [25]:
pearsSim(myMat[:,0], myMat[:,4])

0.30035408702896593

In [26]:
pearsSim(myMat[:,0], myMat[:,0])

1.0

# 2. 협업 필터링

## 2.1 유사한 사용자 찾기

- 협업 필터링: 새로운 객체를 결정하기 위해 데이터셋에 포함된 객체 간의 패턴을 식별 프로세스
- dataset에서 유사한 사용자를 찾아 추천
- 데이터셋에 포함된 다른 사용자들의 선호도(preference) 정보를 활용함으로써 여러 사용자가 '협업'해 동시에 무엇을 추천하는 듯한 효과
- 추천하지 않을 아이템을 filtering
- 두 사람이 이전에 본 영화들에 대해 비슷한 평점을 주었다면 두 사람은 유사도가 높음 
  => 새로운 영화에 대해서도 비슷한 평점을 줄 것으로 기대
- 유클리드점수/피어슨상관계수를 이용하여 데이터셋에서 유사한 사용자를 찾음

In [27]:
import json
import numpy as np

In [28]:
# Compute the Pearson correlation score between user1 and user2 
def pearson_score(dataset, user1, user2):
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 두 사용자에게서 평점을 받은 영화 저장
    common_movies = {}

    # 두 사용자에게서 모두 평점을 받은 영화 찾음
    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    num_ratings = len(common_movies) 

    # 사용자간 공통 영화가 없으면, score = 0 
    if num_ratings == 0:
        return 0

    # 사용자별로 공통으로 평가한 영화의 평점 합 계산 
    user1_sum = np.sum([dataset[user1][item] for item in common_movies])
    user2_sum = np.sum([dataset[user2][item] for item in common_movies])

    # 사용자별로 공통으로 평가한 영화의 평점의 제곱 합 계산
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in common_movies])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in common_movies])

    # 두 사용자가 평가한 영화 평점의 곱
    sum_of_products = np.sum([dataset[user1][item] * dataset[user2][item] for item in common_movies])

    # Calculate the Pearson correlation score
    Sxy = sum_of_products - (user1_sum * user2_sum / num_ratings)
    Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
    Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
    
    if Sxx * Syy == 0:
        return 0

    return Sxy / np.sqrt(Sxx * Syy)

In [29]:
# 입력 user와 유사한 상위 user(num_users)를 찾음
def find_similar_users(dataset, user, num_users):
    if user not in dataset:
        raise TypeError('Cannot find ' + user + ' in the dataset')

    # 입력 사용자와 다른 모든 사용자 간의 Pearson score 계산
    scores = np.array([[x, pearson_score(dataset, user, x)] for x in dataset if x != user])

    # 내림차순으로 정렬
    scores_sorted = np.argsort(scores[:, 1])[::-1]

    # 상위 num_users 추출
    top_users = scores_sorted[:num_users] 

    return scores[top_users] 

In [30]:
ratings_file = 'ratings.json'
user = 'Bill Duffy'

with open(ratings_file, 'r') as f:
    data = json.loads(f.read())
data

{'David Smith': {'Vertigo': 4,
  'Scarface': 4.5,
  'Raging Bull': 3.0,
  'Goodfellas': 4.5,
  'The Apartment': 1.0},
 'Brenda Peterson': {'Vertigo': 3.0,
  'Scarface': 1.5,
  'Raging Bull': 1.0,
  'Goodfellas': 2.0,
  'The Apartment': 5.0,
  'Roman Holiday': 4.5},
 'Bill Duffy': {'Vertigo': 4.5,
  'Scarface': 5.0,
  'Goodfellas': 4.5,
  'The Apartment': 1.0},
 'Samuel Miller': {'Scarface': 3.5,
  'Raging Bull': 5.0,
  'The Apartment': 1.0,
  'Goodfellas': 5.0,
  'Roman Holiday': 1.0},
 'Julie Hammel': {'Scarface': 2.5, 'Roman Holiday': 4.5, 'Goodfellas': 3.0},
 'Clarissa Jackson': {'Vertigo': 5.0,
  'Scarface': 4.5,
  'Raging Bull': 4.0,
  'Goodfellas': 2.5,
  'The Apartment': 1.0,
  'Roman Holiday': 1.5},
 'Adam Cohen': {'Vertigo': 3.5,
  'Scarface': 3.0,
  'The Apartment': 1.0,
  'Goodfellas': 4.5,
  'Roman Holiday': 3.0},
 'Chris Duncan': {'The Apartment': 1.5, 'Raging Bull': 4.5}}

In [31]:
print('\nUsers similar to ' + user + ':\n')
similar_users = find_similar_users(data, user, 3) 
print('User\t\t\tSimilarity score')
print('-'*41)
for item in similar_users:
    print(item[0], '\t\t', round(float(item[1]), 2))


Users similar to Bill Duffy:

User			Similarity score
-----------------------------------------
David Smith 		 0.99
Samuel Miller 		 0.88
Adam Cohen 		 0.86


In [32]:
user = 'Clarissa Jackson'
print('\nUsers similar to ' + user + ':\n')
similar_users = find_similar_users(data, user, 3) 
print('User\t\t\tSimilarity score')
print('-'*41)
for item in similar_users:
    print(item[0], '\t\t', round(float(item[1]), 2))


Users similar to Clarissa Jackson:

User			Similarity score
-----------------------------------------
Chris Duncan 		 1.0
Bill Duffy 		 0.83
Samuel Miller 		 0.73


## 2.2 영화 추천 시스템 만들기

- 특정 사용자에게 영화를 추천할 때 데이터셋에서 유사한 사용자를 찾고 이 사용자의 영화 평점 정보 이용

In [34]:
# 입력 사용자에게 영화를 추천
def get_recommendations(dataset, input_user):
    if input_user not in dataset:
        raise TypeError('Cannot find ' + input_user + ' in the dataset')

    overall_scores = {}
    similarity_scores = {}

    # 입력 user와 dataset의 다른 사용자 간의 유사도계산(pearson상관계수)
    for user in [x for x in dataset if x != input_user]:
        similarity_score = pearson_score(dataset, input_user, user)

        if similarity_score <= 0:
            continue
        
        # 다른 사용자가 본 영화 중에서 입력 user가 보지 않았거나 평점이 0인 영화 list
        filtered_list = [x for x in dataset[user] if x not in \
                dataset[input_user] or dataset[input_user][x] == 0]

        # 유사도에 따라 가중 평점
        for item in filtered_list: 
            overall_scores.update({item: dataset[user][item] * similarity_score})
            similarity_scores.update({item: similarity_score})

    if len(overall_scores) == 0:
        return ['No recommendations possible']

    # 가중 점수에 따라 점수를 정규화(normalization)
    movie_scores = np.array([[score/similarity_scores[item], item] 
            for item, score in overall_scores.items()])

    # Sort in decreasing order 
    movie_scores = movie_scores[np.argsort(movie_scores[:, 0])[::-1]]

    # Extract the movie recommendations
    movie_recommendations = [movie for _, movie in movie_scores]

    return movie_recommendations

In [35]:
ratings_file = 'ratings.json'
user = 'Chris Duncan'

with open(ratings_file, 'r') as f:
    data = json.loads(f.read())

In [36]:
print("\nMovie recommendations for " + user + ":")
movies = get_recommendations(data, user) 
for i, movie in enumerate(movies):
    print(str(i+1) + '. ' + movie)


Movie recommendations for Chris Duncan:
1. Vertigo
2. Scarface
3. Goodfellas
4. Roman Holiday


In [37]:
user = 'Julie Hammel'
print("\nMovie recommendations for " + user + ":")
movies = get_recommendations(data, user) 
for i, movie in enumerate(movies):
    print(str(i+1) + '. ' + movie)


Movie recommendations for Julie Hammel:
1. The Apartment
2. Vertigo
3. Raging Bull


### MovieLens 영화 평점 데이터로 테스트